### **Ice Marketing Campaign Strategy 2017**

#### **1. Introduction**
- The objective of this product is to develop an advertising strategy for projected winners in the video game sector.
- Historical video game sales data will be analyzed to inform the forecast and identify potential big winners that should garner advertising support proportional to their sales.
- The dataset is the aggregation of several open sources providing insights on user and expert reviews, platforms, and historical sales data.

#### **2. Approach**
The dataset will be analyzed to understand the following:
- Sales Performance Across Different Platforms at Different Points in Time
- Impact of User and Professional Reviews on Sales
- Profitability Across Different Genres
- Regional User Profiles

The project will consist of the following stages:
1. Data Preparation
2. Data Analysis
3. Regional User Profile Definition
4. Hypothesis Testing
5. Conclusion

#### **3. Initialization**
Importing all relevant libraries and loading in the dataset.

In [69]:
import pandas as pd
import numpy as np
import math as mt
import random as rd
from matplotlib import pyplot as pyplot
from scipy import stats as st

In [70]:
#Load the data

url = "https://raw.githubusercontent.com/Flamingo-Rocker/Video-Game-Marketing-Strategy/refs/heads/main/games.csv"
games = pd.read_csv(url)

#Display first few rows the check that the pull worked
games.head()

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


#### **4. Data Preparation**
- Replace column names
- Convert data to required types
- Address missing and/or duplicate values as-necessary
- Enrich the data with additional columns useful for analysis

In [71]:
#Print the general/summary information about the dataset
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


In [72]:
#Rename the columns to appropriate style
games = games.rename(
    columns={
        'Name': 'name',
        'Platform': 'platform',
        'Year_of_Release': 'year_of_release',
        'Genre': 'genre',
        'NA_sales': 'na_sales',
        'EU_sales': 'eu_sales',
        'JP_sales': 'jp_sales',
        'Other_sales': 'other_sales',
        'Critic_Score': 'critic_score',
        'User_Score': 'user_score',
        'Rating': 'rating'
    }
)
#print(games.columns)

In [73]:
#Verify the number of duplicated rows and remove if necessary
games.duplicated().sum()

0

There are no duplicate rows - missing values shall now be assessed.

In [74]:
#Review number of missing values
print(games.isna().sum())
#print()

#Drop rows where name or year_of_release are missing
games.dropna(subset=['name', 'year_of_release'], inplace=True)

#Replace missing critic scores with the median critic score
games['critic_score'] = games['critic_score'].fillna(games['critic_score'].median())

#Replace 'tbd' values with NaN in user_score
games['user_score'] = games['user_score'].replace({'tbd': np.nan})

#Convert user_score to float and replace missing values with median score
games['user_score'] = games['user_score'].astype('float64')
games['user_score'] = games['user_score'].fillna(games['user_score'].median())

#Replace missing values in rating column with "No Rating"
games['rating'] = games['rating'].fillna('No Rating')

#Change year_of_release to an integer for cleaner optics
games['year_of_release'] = games['year_of_release'].astype('int64')

#Verify missing values resolved
#print(games.isna().sum())

name                  2
platform              0
year_of_release     269
genre                 2
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8578
user_score         6701
rating             6766
dtype: int64


Several steps were taken above to address issues with datatypes and missing values:
- A small number of missing values were detected in the **`name`** and  **`year_of_release`** columns
    - The affected rows were dropped as they would not significantly impact the available data
- There was a significant amount of missing values in the following columns:
    - **`critic_score`**: 
        - Nearly half of the rows were missing data in this column
        - The median critic score was used to fill in missing values to prevent data quality loss
    - **`user_score`**:
        - This column was an **object** datatype and needed to be converted to **float**, but the presence of the 'tbd' string in the column needed to be addressed first
        - To facilitate the conversion, the 'tbd's were replaced with **nan** values
        - The column was then converted to a **float** datatype, and missing values were then replaced with the median user score to prevent data quality loss
    - **`rating`**:
        - Missing values in this column were replaced with the string 'No Rating to prevent data quality loss
    - **`year_of_release`**:
        - Converted to an **int** datatype as years are discrete values adn the decimal is unnecessary
